In [2]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
orders = pd.read_excel('superstore_sales.xlsx', sheet_name="Orders")
people = pd.read_excel('superstore_sales.xlsx', sheet_name="People")
returns = pd.read_excel('superstore_sales.xlsx', sheet_name="Returns")

In [4]:
df = orders.copy()

In [5]:
df.head(3)

,order_id,order_date,ship_date,ship_mode,customer_name,segment,state,country,market,region,...,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,year
0,AG-2011-2040,2011-01-01,2011-01-06,Standard Class,Toby Braunhardt,Consumer,Constantine,Algeria,Africa,Africa,...,Office Supplies,Storage,"Tenex Lockers, Blue",408.300,2,0.0,106.140,35.46,Medium,2011
1,IN-2011-47883,2011-01-01,2011-01-08,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,Oceania,...,Office Supplies,Supplies,"Acme Trimmer, High Speed",120.366,3,0.1,36.036,9.72,Medium,2011
2,HU-2011-1220,2011-01-01,2011-01-05,Second Class,Annie Thurman,Consumer,Budapest,Hungary,EMEA,EMEA,...,Office Supplies,Storage,"Tenex Box, Single Width",66.120,4,0.0,29.640,8.17,High,2011


In [6]:
df.shape

(51290, 21)

Check for missing values.

In [7]:
df.isnull().sum()

order_id          0
order_date        0
ship_date         0
ship_mode         0
customer_name     0
segment           0
state             0
country           0
market            0
region            0
product_id        0
category          0
sub_category      0
product_name      0
sales             0
quantity          0
discount          0
profit            0
shipping_cost     0
order_priority    0
year              0
dtype: int64

In [8]:
df.describe()

,sales,quantity,discount,profit,shipping_cost,year
count,51290.000000,51290.000000,51290.000000,51290.000000,51290.000000,51290.000000
mean,246.490581,3.476545,0.142908,28.641740,26.375818,2012.777208
std,487.565361,2.278766,0.212280,174.424113,57.296810,1.098931
min,0.444000,1.000000,0.000000,-6599.978000,0.002000,2011.000000
25%,30.758625,2.000000,0.000000,0.000000,2.610000,2012.000000
50%,85.053000,3.000000,0.000000,9.240000,7.790000,2013.000000
75%,251.053200,5.000000,0.200000,36.810000,24.450000,2014.000000
max,22638.480000,14.000000,0.850000,8399.976000,933.570000,2014.000000


**1\. PRODUCTS**

What product has the most sales

In [9]:
# Check the number of distinct products available
count = 0
distinct = []

for i in range(0, len(df['product_name'])) :
    if df['product_name'][i] not in distinct:
        distinct.append(df['product_name'][i])
        count += 1
print("Dataset has ", count, "distinct products")
#Alternatively
# print(len(pd.unique(df['product_name'])))

Dataset has  3788 distinct products


In [10]:
# Get the products with the most sales
# MSP = Most Sold Products
msp = df.groupby('product_name').agg({'quantity': 'sum'}).sort_values('quantity', ascending=False)
topten = msp.head(10)
topten

,quantity
product_name,
Staples,876
"Cardinal Index Tab, Clear",337
"Eldon File Cart, Single Width",321
"Rogers File Cart, Single Width",262
"Sanford Pencil Sharpener, Water Color",259
"Stockwell Paper Clips, Assorted Sizes",253
"Avery Index Tab, Clear",252
"Ibico Index Tab, Clear",251
"Smead File Cart, Single Width",250


LET US CHECK HOW MANY PRODUCTS HAVE SIMILAR NAME

In [11]:
sim = [col for col in df['product_name'] if 'Index Tab' in col]
# To check any other similar names , switch the Index Tab with any column of your choice
count = 0
new = []
for i in range(0, len(sim)):
    if sim[i] not in new :
        new.append(sim[i])
        count += 1
print(count)
print(new)

# Alternatively

# bb=dict(zip(list(sim),[list(sim).count(i) for i in list(sim)]))
# print(bb)

17
['Ibico Index Tab, Durable', 'Avery Index Tab, Economy', 'Avery Index Tab, Clear', 'Ibico Index Tab, Clear', 'Cardinal Index Tab, Economy', 'Cardinal Index Tab, Clear', 'Avery Index Tab, Durable', 'Acco Index Tab, Clear', 'Wilson Jones Index Tab, Economy', 'Acco Index Tab, Durable', 'Cardinal Index Tab, Durable', 'Ibico Index Tab, Economy', 'Wilson Jones Index Tab, Recycled', 'Wilson Jones Index Tab, Durable', 'Acco Index Tab, Economy', 'Wilson Jones Index Tab, Clear', 'Computer Printout Index Tabs']


In [12]:
# Products with the most profut
# mpp =  most profitable products

mpp = df.groupby('product_name').agg({'profit' : 'sum'})

mpp.nlargest(5,'profit')



,profit
product_name,
Canon imageCLASS 2200 Advanced Copier,25199.9280
"Cisco Smart Phone, Full Size",17238.5206
"Motorola Smart Phone, Full Size",17027.1130
"Hoover Stove, Red",11807.9690
"Sauder Classic Bookcase, Traditional",10672.0730


In [13]:
filtered = df.copy()
diff_date = (df['ship_date'] - df['order_date'])/ np.timedelta64(1, 'D')


In [14]:
filtered['diff_date'] = diff_date

In [15]:
filt = filtered.pop('diff_date')
filtered.insert(3, 'diff_date', filt)


In [16]:
filtered.head(5)

,order_id,order_date,ship_date,diff_date,ship_mode,customer_name,segment,state,country,market,...,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,year
0,AG-2011-2040,2011-01-01,2011-01-06,5.0,Standard Class,Toby Braunhardt,Consumer,Constantine,Algeria,Africa,...,Office Supplies,Storage,"Tenex Lockers, Blue",408.300,2,0.0,106.140,35.46,Medium,2011
1,IN-2011-47883,2011-01-01,2011-01-08,7.0,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,...,Office Supplies,Supplies,"Acme Trimmer, High Speed",120.366,3,0.1,36.036,9.72,Medium,2011
2,HU-2011-1220,2011-01-01,2011-01-05,4.0,Second Class,Annie Thurman,Consumer,Budapest,Hungary,EMEA,...,Office Supplies,Storage,"Tenex Box, Single Width",66.120,4,0.0,29.640,8.17,High,2011
3,IT-2011-3647632,2011-01-01,2011-01-05,4.0,Second Class,Eugene Moren,Home Office,Stockholm,Sweden,EU,...,Office Supplies,Paper,"Enermax Note Cards, Premium",44.865,3,0.5,-26.055,4.82,High,2011
4,IN-2011-47883,2011-01-01,2011-01-08,7.0,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,...,Furniture,Furnishings,"Eldon Light Bulb, Duo Pack",113.670,5,0.1,37.770,4.70,Medium,2011


In [17]:
filtered['month'] = pd.DatetimeIndex(filtered['order_date']).month 

In [18]:
filtered['month'] = filtered.month.apply(lambda x : str(int(x)).zfill(2))

In [19]:
filtered.head(2)

,order_id,order_date,ship_date,diff_date,ship_mode,customer_name,segment,state,country,market,...,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,year,month
0,AG-2011-2040,2011-01-01,2011-01-06,5.0,Standard Class,Toby Braunhardt,Consumer,Constantine,Algeria,Africa,...,Storage,"Tenex Lockers, Blue",408.300,2,0.0,106.140,35.46,Medium,2011,01
1,IN-2011-47883,2011-01-01,2011-01-08,7.0,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,...,Supplies,"Acme Trimmer, High Speed",120.366,3,0.1,36.036,9.72,Medium,2011,01


In [20]:
# stt = ship_mode to time
stt = filtered.groupby('ship_mode').agg({'diff_date' : 'mean'}).sort_values('diff_date', ascending = False)

In [21]:
stt = stt['diff_date'].round(2)
stt

ship_mode
Standard Class    5.00
Second Class      3.23
First Class       2.18
Same Day          0.04
Name: diff_date, dtype: float64

In [22]:
# stp = ship_mode to profit
stp = filtered.groupby('ship_mode').agg({'profit' : 'sum'}).sort_values('profit', ascending= False)

In [23]:
stp = stp['profit'].round()
stp

ship_mode
Standard Class    890596.0
Second Class      294161.0
First Class       208105.0
Same Day           76173.0
Name: profit, dtype: float64

Let us see how the shipping mode affects the delivery and the profit made

In [24]:
ship = pd.merge(stt, stp, how= 'inner', on= 'ship_mode')

In [25]:
ship

,diff_date,profit
ship_mode,,
Standard Class,5.00,890596.0
Second Class,3.23,294161.0
First Class,2.18,208105.0
Same Day,0.04,76173.0


In [26]:
mss = df.groupby('segment').agg({'quantity': 'sum'}).sort_values('quantity', ascending=False)

In [27]:
mss

,quantity
segment,
Consumer,92157
Corporate,53565
Home Office,32590


In [28]:
mps = df.groupby('segment').agg({'profit': 'sum'}).sort_values('profit', ascending=False)

In [29]:
mps.nlargest(3, 'profit')

,profit
segment,
Consumer,749239.78206
Corporate,442785.85866
Home Office,277009.18056


In [30]:
msr = df.groupby('region').agg({'quantity': 'sum'}).sort_values('region', ascending=False)

In [31]:
msr.shape

(13, 1)

In [32]:
msr.nlargest(13, 'quantity')

,quantity
region,
Central,41762
South,25206
North,18021
Oceania,12838
West,12266
Southeast Asia,11822
EMEA,11517
East,10618
Africa,10564


In [33]:
mpr = df.groupby('region').agg({'profit': 'sum'}).sort_values('profit', ascending=False)

In [34]:
mpr.nlargest(13, 'profit')

,profit
region,
Central,311403.98164
North,194597.95252
North Asia,165578.42100
South,140355.76618
Central Asia,132480.18700
Oceania,121666.64200
West,108418.44890
East,91522.78000
Africa,88871.63100


In [35]:
msm = df.groupby('market').agg({'quantity': 'sum'}).sort_values('quantity', ascending=False)

In [36]:
msm.nlargest(7, 'quantity')

,quantity
market,
APAC,41226
LATAM,38526
US,37873
EU,37773
EMEA,11517
Africa,10564
Canada,833


In [37]:
mpm = df.groupby('market').agg({'profit': 'sum'}).sort_values('profit', ascending=False)

In [38]:
mpm.nlargest(13, 'profit')

,profit
market,
APAC,437577.57900
EU,372829.74150
US,286397.02170
LATAM,221643.48708
Africa,88871.63100
EMEA,43897.97100
Canada,17817.39000


In [39]:
# Get the sum of all products
new_sum = pd.merge(pd.merge(df, msp, on='product_name'), mpp, on='product_name')

In [40]:
new_sum = new_sum.drop(['quantity_x', 'profit_x'], axis= 1)

In [41]:
new_sum.rename(columns={'profit_y' : 'profit', 'quantity_y' : 'quantity'}, inplace = True)

In [42]:
new_sum.columns

Index(['order_id', 'order_date', 'ship_date', 'ship_mode', 'customer_name',
       'segment', 'state', 'country', 'market', 'region', 'product_id',
       'category', 'sub_category', 'product_name', 'sales', 'discount',
       'shipping_cost', 'order_priority', 'year', 'quantity', 'profit'],
      dtype='object')

In [43]:
new_sum = df.drop_duplicates(subset=['product_name'])

In [44]:
segment_products = new_sum[['segment','product_name', 'sales']]

In [45]:
segment_products.nsmallest(5, 'sales')

,segment,product_name,sales
1349,Consumer,Avery Round Ring Poly Binders,0.852
5654,Corporate,Acco 3-Hole Punch,0.876
3718,Corporate,Acco Economy Flexible Poly Round Ring Binder,1.044
674,Consumer,Wilson Jones Easy Flow II Sheet Lifters,1.080
412,Consumer,"Avery Hole Reinforcements, Clear",1.161
